In [1]:
# Cell 2: Download Citi Bike Data for 2024
# Purpose: Fetch monthly ZIP files for 2024 containing trip data
import requests
import os

def download_citibike_data(year, month):
    month_str = f"{month:02d}"
    #url = f"https://s3.amazonaws.com/tripdata/JC-{year}{month_str}-citibike-tripdata.zip"
    url = f"https://s3.amazonaws.com/tripdata/{year}{month_str}-citibike-tripdata.csv.zip" 
    zip_file_path = os.path.join( "raw_data","2024", f"{year}{month_str}-citibike-tripdata.csv.zip")
    
    if os.path.exists(zip_file_path):
        print(f"File for {year}-{month_str} already exists")
        return zip_file_path
    
    print(f"Downloading data for {year}-{month_str}...")
    response = requests.get(url)
    response.raise_for_status()
    with open(zip_file_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {zip_file_path}")
    return zip_file_path

# Download data for 2024 (12 months)
year = 2024
zip_paths = [download_citibike_data(year, month) for month in range(1, 13)]
print("All data downloaded successfully")

File for 2024-01 already exists
File for 2024-02 already exists
File for 2024-03 already exists
File for 2024-04 already exists
File for 2024-05 already exists
File for 2024-06 already exists
File for 2024-07 already exists
File for 2024-08 already exists
File for 2024-09 already exists
File for 2024-10 already exists
File for 2024-11 already exists
File for 2024-12 already exists
All data downloaded successfully


In [2]:
# Cell 2: Download Citi Bike Data for 2024
# Purpose: Fetch monthly ZIP files for 2024 containing trip data


In [3]:
import os
import zipfile
import pandas as pd

# Set paths
folder_path = "raw_data/2024"
output_folder = "filter_data_2024"
os.makedirs(output_folder, exist_ok=True)

# File list
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".zip")])

# Track column structure
column_set = None
matching_files = []
null_counts = {}
row_counts = {}

# Process ZIPs
for file_name in file_list:
    month = file_name.split("-")[0]
    file_path = os.path.join(folder_path, file_name)
    monthly_df_list = []

    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        csv_files = [f for f in zip_ref.namelist() if f.endswith(".csv") and "__MACOSX" not in f]
        if not csv_files:
            print(f"⚠️ No CSVs in {file_name}")
            continue

        for csv_name in csv_files:
            with zip_ref.open(csv_name) as csv_file:
                try:
                    df = pd.read_csv(csv_file, low_memory=False)

                    # Drop index column if present
                    if "Unnamed: 0" in df.columns:
                        df.drop(columns=["Unnamed: 0"], inplace=True)

                    # Drop any column with "duplicate" in the name
                    dup_cols = [col for col in df.columns if "duplicate" in col.lower()]
                    if dup_cols:
                        print(f"⚠️ Dropping duplicate columns in {csv_name}: {dup_cols}")
                        df.drop(columns=dup_cols, inplace=True)

                    # Check column consistency
                    if column_set is None:
                        column_set = set(df.columns)
                        matching_files.append(file_name)
                    elif set(df.columns) == column_set:
                        matching_files.append(file_name)
                    else:
                        print(f"❌ {file_name} → {csv_name} has different columns, skipping.")
                        continue

                    # Save raw stats
                    row_counts[csv_name] = len(df)
                    null_counts[csv_name] = df.isnull().sum().to_dict()

                    monthly_df_list.append(df)

                except Exception as e:
                    print(f"❌ Failed to process {csv_name} in {file_name}: {e}")

    # Combine and save cleaned data
    if monthly_df_list:
        combined_df = pd.concat(monthly_df_list, ignore_index=True)
        cleaned_df = combined_df.dropna()

        output_csv_name = f"{month}_filtered.csv"
        output_path = os.path.join(output_folder, output_csv_name)
        cleaned_df.to_csv(output_path, index=False)

        print(f"✅ Saved: {output_csv_name} with {len(cleaned_df)} cleaned rows")

# Summary
print("\n--- Summary ---")
print("\nFiles with matching columns:")
print(matching_files)

print("\nRow counts per original CSV file:")
for file, count in row_counts.items():
    print(f"{file}: {count} rows")

print("\nNull values per original CSV file:")
for file, counts in null_counts.items():
    print(f"\n{file}:")
    for col, val in counts.items():
        print(f"  {col}: {val}")


✅ Saved: 202401_filtered.csv with 1881977 cleaned rows
✅ Saved: 202402_filtered.csv with 2115989 cleaned rows
✅ Saved: 202403_filtered.csv with 2656084 cleaned rows
✅ Saved: 202404_filtered.csv with 3209060 cleaned rows
✅ Saved: 202405_filtered.csv with 4220136 cleaned rows
✅ Saved: 202406_filtered.csv with 4768748 cleaned rows
✅ Saved: 202407_filtered.csv with 4707631 cleaned rows
⚠️ Dropping duplicate columns in 202408-citibike-tripdata_3.csv: ['rideable_type_duplicate_column_name_1']
⚠️ Dropping duplicate columns in 202408-citibike-tripdata_2.csv: ['rideable_type_duplicate_column_name_1']
⚠️ Dropping duplicate columns in 202408-citibike-tripdata_1.csv: ['rideable_type_duplicate_column_name_1']
⚠️ Dropping duplicate columns in 202408-citibike-tripdata_5.csv: ['rideable_type_duplicate_column_name_1']
⚠️ Dropping duplicate columns in 202408-citibike-tripdata_4.csv: ['rideable_type_duplicate_column_name_1']
✅ Saved: 202408_filtered.csv with 4587868 cleaned rows
✅ Saved: 202409_filtered.

In [4]:
import os
import pandas as pd

# Set the folder where your .csv files are located
folder_path = "filter_data_2024"
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])

column_set = None
matching_files = []
null_counts = {}
row_counts = {}

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    try:
        df = pd.read_csv(file_path)

        # Initialize or compare column set
        if column_set is None:
            column_set = set(df.columns)
            matching_files.append(file_name)
        elif set(df.columns) == column_set:
            matching_files.append(file_name)
        else:
            print(f"❌ {file_name} has different columns")

        # Count null values per column
        null_counts[file_name] = df.isnull().sum().to_dict()

        # Count number of rows
        row_counts[file_name] = len(df)

    except Exception as e:
        print(f"⚠️ Error reading {file_name}: {e}")

# Display summary
print("\n✅ Files with matching columns:")
print(matching_files)

print("\n📊 Row counts per file:")
for file, count in row_counts.items():
    print(f"{file}: {count} rows")

print("\n🧼 Null values per file:")
for file, counts in null_counts.items():
    print(f"\n{file}:")
    for col, val in counts.items():
        print(f"  {col}: {val}")


C:\Users\Jaath\AppData\Local\Temp\ipykernel_43744\2739639094.py:16: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\Jaath\AppData\Local\Temp\ipykernel_43744\2739639094.py:16: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\Jaath\AppData\Local\Temp\ipykernel_43744\2739639094.py:16: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\Jaath\AppData\Local\Temp\ipykernel_43744\2739639094.py:16: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\Jaath\AppData\Local\Temp\ipykernel_43744\2739639094.py:16: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)



✅ Files with matching columns:
['202401_filtered.csv', '202402_filtered.csv', '202403_filtered.csv', '202404_filtered.csv', '202405_filtered.csv', '202406_filtered.csv', '202407_filtered.csv', '202408_filtered.csv', '202409_filtered.csv', '202410_filtered.csv', '202411_filtered.csv', '202412_filtered.csv']

📊 Row counts per file:
202401_filtered.csv: 1881977 rows
202402_filtered.csv: 2115989 rows
202403_filtered.csv: 2656084 rows
202404_filtered.csv: 3209060 rows
202405_filtered.csv: 4220136 rows
202406_filtered.csv: 4768748 rows
202407_filtered.csv: 4707631 rows
202408_filtered.csv: 4587868 rows
202409_filtered.csv: 4981399 rows
202410_filtered.csv: 5132716 rows
202411_filtered.csv: 3697286 rows
202412_filtered.csv: 2302930 rows

🧼 Null values per file:

202401_filtered.csv:
  ride_id: 0
  rideable_type: 0
  started_at: 0
  ended_at: 0
  start_station_name: 0
  start_station_id: 0
  end_station_name: 0
  end_station_id: 0
  start_lat: 0
  start_lng: 0
  end_lat: 0
  end_lng: 0
  memb

In [ ]:
# import os
# import pandas as pd

# def combine_monthly_csvs(folder_path: str, output_file: str) -> None:
#     """
#     Combines all CSV files from the folder into a single CSV file.
    
#     Args:
#         folder_path (str): Directory containing monthly CSV files.
#         output_file (str): Output path for the combined CSV file.
#     """
#     all_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
#     combined_df = []

#     for file in sorted(all_files):
#         file_path = os.path.join(folder_path, file)
#         print(f"Loading: {file}")
#         df = pd.read_csv(file_path, low_memory=False)
#         combined_df.append(df)

#     # Concatenate all and write to a single CSV
#     result = pd.concat(combined_df, ignore_index=True)
#     result.to_csv(output_file, index=False)
#     print(f"✅ Combined CSV saved as: {output_file}")

# # 👇 Run the combiner
# combine_monthly_csvs("filter_data_2024", "all_months_combined.csv")



In [ ]:
# import pandas as pd

# # Load the combined dataset
# file_path = r"./filter_data_2024/all_months_combined.csv"
# df = pd.read_csv(file_path, usecols=["start_station_id"])

# # Drop missing station IDs and ensure string type
# df = df.dropna(subset=["start_station_id"])
# df["start_station_id"] = df["start_station_id"].astype(str)

# # Find top 3 most frequent start stations
# top_3_stations = (
#     df["start_station_id"]
#     .value_counts()
#     .head(3)
#     .rename_axis("start_station_id")
#     .reset_index(name="trip_count")
# )


# print("Top 3 Start Stations by Trip Count:")
# print(top_3_stations)


In [ ]:
# import pandas as pd

# # === 1. Load the combined dataset ===
# file_path = r"D:\Code\Git\ESDS_500_Final\all_months_combined.csv"
# df = pd.read_csv(file_path)

# # === 2. Drop missing start station IDs and convert to string ===
# df = df.dropna(subset=["start_station_id"])
# df["start_station_id"] = df["start_station_id"].astype(str)

# # === 3. Find top 3 most frequent start stations ===
# top_3_stations = (
#     df["start_station_id"]
#     .value_counts()
#     .head(3)
#     .rename_axis("start_station_id")
#     .reset_index(name="trip_count")
# )

# # === 4. Extract top station IDs into a list ===
# top_station_ids = top_3_stations["start_station_id"].tolist()

# # === 5. Filter original DataFrame for only top 3 start stations ===
# df_top_stations = df[df["start_station_id"].isin(top_station_ids)]

# # === 6. Save the filtered data to a new CSV file ===
# output_path = r"D:\Code\Git\ESDS_500_Final\BikeRide2024Top3Location.csv"
# df_top_stations.to_csv(output_path, index=False)

# print(f"✅ Filtered data for top 3 stations saved to: {output_path}")
